In [ ]:
!git clone https://github.com/yueqingliang1/UNBench.git

Cloning into 'UNBench'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 204 (delta 44), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (204/204), 8.47 MiB | 21.53 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
%cd UNBench
!pip install -r requirements.txt

/content/UNBench/UNBench


In [ ]:
!pip install langchain-openai
from google.colab import userdata
from langchain_openai import ChatOpenAI

api_key = userdata.get("DEEPSEEK_API_KEY")

llm = ChatOpenAI(
    model="deepseek-chat",
    api_key=api_key,
    base_url="https://api.deepseek.com",
    temperature=0,
)

In [ ]:
import json
with open("data/task1.json") as f:
    test = json.load(f)

In [ ]:
draft_ids = []
authors = []
coauthors = []
choices_2 = []
choices_3 = []
choices_4 = []
choices_5 = []

for instance in test:
    draft_ids.append(instance['folder_id'])
    authors.append(instance['author'])
    coauthors.append(instance['coauthor'])
    choices_2.append(instance['choices_2'])
    choices_3.append(instance['choices_3'])
    choices_4.append(instance['choices_4'])
    choices_5.append(instance['choices_5'])

In [ ]:
import os
path = 'data/task1'
drafts = []
for i in draft_ids:
    folder_path = os.path.join(path, str(i))
    files = os.listdir(folder_path)
    json_file = [file for file in files if file.endswith('EN.json')][0]
    with open(os.path.join(folder_path, json_file)) as f:
        draft = json.load(f)
    drafts.append(draft['Content'])

In [ ]:
import random
from tqdm import tqdm

def run_task3_k_choices(client, drafts, authors, choices_k):
    results = []
    invalid_responses = []
    for i, (draft, author, choice_k) in tqdm(enumerate(zip(drafts, authors, choices_k))):
        system_prompt = f"""You are a senior diplomat representing {author} at the United Nations.

Examples of good choices:
- If draft focuses on climate change → choose a country with strong environmental policies
- If draft focuses on security → choose a permanent Security Council member
- If draft focuses on regional issues → choose a country from that region

CRITICAL RULES:
1. You MUST choose EXACTLY ONE country from the provided list
2. Your response MUST be ONLY the country name - nothing else
3. Do NOT suggest groups, alliances, or organizations
4. Base your choice on the draft resolution content"""

        user_prompt = f"""{author} is drafting a resolution to submit to the United Nations Security Council and is seeking a coauthor.

        Review the following draft resolution and choose a coauthor from the list below.
        Respond only with the name of the chosen country and provide no additional explanation.

        Available coauthors: {', '.join(choice_k)}

        Draft Resolution:
        {draft}

        Answer:
        """

        response = client.chat.completions.create(
            model=your_model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=20,
            temperature=0.0
        )
        result = response.choices[0].message.content.strip()
        valid_response = choice_k
        if result not in valid_response:
            print(f'Invalid response: {result}')
            result = random.choice(valid_response)
            invalid_responses.append(i)
        results.append(result)
    return results, invalid_responses

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com"
)
your_model_name = "deepseek-chat"

In [ ]:
results_2, invalid_responses_2 = run_task3_k_choices(client, drafts, authors, choices_2)

30it [00:50,  1.67s/it]


In [ ]:
results_3, invalid_responses_3 = run_task3_k_choices(client, drafts, authors, choices_3)

30it [00:49,  1.66s/it]


In [ ]:
results_4, invalid_responses_4 = run_task3_k_choices(client, drafts, authors, choices_4)

13it [00:20,  1.67s/it]

Invalid response: Egypt


28it [00:44,  1.51s/it]

Invalid response: Egypt


30it [00:48,  1.61s/it]


In [ ]:
results_5, invalid_responses_5 = run_task3_k_choices(client, drafts, authors, choices_5)

30it [00:49,  1.66s/it]


In [ ]:
def calculate_accuracy(results, ground_truth):
    correct = 0
    for i in range(len(results)):
        if results[i] == ground_truth[i]:
            correct += 1
    return correct/len(results)

In [ ]:
accuracy_2 = calculate_accuracy(results_2, coauthors)
accuracy_3 = calculate_accuracy(results_3, coauthors)
accuracy_4 = calculate_accuracy(results_4, coauthors)
accuracy_5 = calculate_accuracy(results_5, coauthors)

In [ ]:
print(f'Accuracy for 2 choices: {accuracy_2}')
print(f'Accuracy for 3 choices: {accuracy_3}')
print(f'Accuracy for 4 choices: {accuracy_4}')
print(f'Accuracy for 5 choices: {accuracy_5}')

Accuracy for 2 choices: 0.5666666666666667
Accuracy for 3 choices: 0.5
Accuracy for 4 choices: 0.3333333333333333
Accuracy for 5 choices: 0.36666666666666664
